In [ ]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd
from typing import Self

# Load data and examples

In [ ]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [ ]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples

# Part A

In [ ]:
todays_examples[0] = todays_examples[0]._replace(input_data="125 17")
todays_examples

In [ ]:
# too early optimisation...
class Stone:
    def __init__(self, val: int, left: Self | None = None, right: Self | None = None):
        self.val = val
        self.left = left
        self.right = right

    def blink(self):
        if self.val == 0:
            self.val = 1
        elif len(str(self.val)) % 2 == 0:
            val_str = str(self.val)
            left_val = int(val_str[: len(val_str) // 2])
            right_val = int(val_str[len(val_str) // 2 :])
            self.val = left_val
            old_right_stone = self.right
            self.right = Stone(right_val, self, old_right_stone)
            if old_right_stone != None:
                old_right_stone.left = self.right
        else:
            self.val *= 2024

    def __repr__(self):
        return f"Stone({str(self.val)})"

In [ ]:
def stone_values_to_stone_list(stone_values: list[int]):
    if len(stone_values) < 1:
        return None
    initial_stone = Stone(stone_values[0])
    curr_stone = initial_stone
    for val in stone_values[1:]:
        curr_stone.right = Stone(val, curr_stone)
        curr_stone = curr_stone.right
    return initial_stone

In [ ]:
def blink_stone_row(initial_stone: Stone):
    curr_stone = initial_stone
    while curr_stone is not None:
        next_stone = curr_stone.right
        curr_stone.blink()
        curr_stone = next_stone
    return initial_stone


def print_stone_row(initial_stone: Stone):
    curr_stone = initial_stone
    while curr_stone is not None:
        print(curr_stone, end=" ")
        curr_stone = curr_stone.right


def count_stone_row(initial_stone: Stone):
    curr_stone = initial_stone
    result = 0
    while curr_stone is not None:
        result += 1
        curr_stone = curr_stone.right
    return result

In [ ]:
def part_a(data: str) -> str:
    stone_values = [int(x) for x in data.split(" ")]
    initial_stone = stone_values_to_stone_list(stone_values)
    for _ in range(25):
        blink_stone_row(initial_stone)
    result = count_stone_row(initial_stone)
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache(maxsize=None)
def num_stones_from_the_stone(stone: int, num_iterations: int):
    result = 1
    for curr_iter in range(num_iterations):
        # rint(f'{curr_iter=}, {stone=}')
        if stone == 0:
            stone = 1
        elif len(str(stone)) % 2 == 0:
            val_str = str(stone)
            # print(val_str,val_str[:len(val_str)//2], len(val_str))
            left_val = int(val_str[: len(val_str) // 2])
            right_val = int(val_str[len(val_str) // 2 :])
            stone = left_val
            result += num_stones_from_the_stone(
                right_val, num_iterations - curr_iter - 1
            )
        else:
            stone *= 2024
    return result

In [ ]:
def even_more_efficient_stone_blinking(stones: list, num_iterations: int):
    result = 0
    for stone in stones:
        result += num_stones_from_the_stone(stone, num_iterations)
    return result

In [ ]:
def part_b(data: str) -> str:
    stone_values = [int(x) for x in data.split(" ")]
    result = even_more_efficient_stone_blinking(stone_values, 75)
    return str(result)

In [ ]:
part_b(data)